In [54]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from utils import calculate_gradient_and_update, cnn_train_step, cnn_val_step, save_checkpoint, mmn_train_step, mmn_val_step
from dataloader import CustomDataloader
from networks import CNNRegression, MultiModalNetwork
import tqdm

#### Automatic GPU optimization, if available

In [55]:
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", cuda_available)

Is CUDA available?  True


In [56]:
if (torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('GPU not available, using CPU')
    

Using GPU: NVIDIA GeForce GTX 1660 Ti


### Model 1 - Linear Regression ###

In [57]:
#load train test and validation data from tensor_collection
cm_x_train = torch.load('tensor_collection\\cm_x_train.pt')
cm_y_train = torch.load('tensor_collection\\cm_y_train.pt')
cm_x_test = torch.load('tensor_collection\\cm_x_test.pt')
cm_y_test = torch.load('tensor_collection\\cm_y_test.pt')
cm_x_val = torch.load('tensor_collection\\cm_x_val.pt')
cm_y_val = torch.load('tensor_collection\\cm_y_val.pt')



#### Hyperparameters setup

In [58]:
num_features = cm_x_train.shape[1]

train_loss_history = []
val_loss_history = []
num_epochs = 100
alpha = 0.1

# Initialize theta to random values between -2 and 2
theta = np.random.uniform(-2, 2, (num_features))


#### Model 1 - Training Loop 

In [59]:

# Training Loop
for t in range(num_epochs):
    # Training step
    train_loss, theta = calculate_gradient_and_update(cm_x_train, cm_y_train, theta, alpha)
    train_loss_history.append(train_loss)

    # Validation step
    val_loss, _ = calculate_gradient_and_update(cm_x_val, cm_y_val, theta, 0)  # alpha=0 to prevent updates
    val_loss_history.append(val_loss)


In [60]:
#save theta and loss history for analysis notebook
theta_torch = torch.tensor(theta, dtype=torch.float32)
train_loss_history_torch = torch.tensor(train_loss_history, dtype=torch.float32)
val_loss_history_torch = torch.tensor(val_loss_history, dtype=torch.float32)

torch.save(theta_torch, 'tensor_collection\\lr_theta.pt')
torch.save(train_loss_history_torch, 'tensor_collection\\lr_train_loss.pt')
torch.save(val_loss_history_torch, 'tensor_collection\\lr_val_loss.pt')


### Model 2 - Training CNN ###

#### Load in Train, Val, and Test sets for Model 2

In [61]:
directory = 'tensor_collection\\'

#load train and val csv files
cnn_x_train = pd.read_csv(directory + 'cnn_x_train.csv')
cnn_x_val = pd.read_csv(directory + 'cnn_x_val.csv')
cnn_y_train_tensor = torch.load(directory + 'cnn_y_train.pt')
cnn_y_val_tensor = torch.load(directory + 'cnn_y_val.pt')

print(type(cnn_x_train))
print(type(cnn_y_train_tensor))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [62]:
print(cnn_x_train.head())

                                            filename
0  data\images\25_0_0_20170117203843296.jpg.chip.jpg
1  data\images\56_0_3_20170105180725694.jpg.chip.jpg
2  data\images\29_1_0_20170120133642078.jpg.chip.jpg
3  data\images\35_1_3_20170112211501949.jpg.chip.jpg
4  data\images\29_1_0_20170116211713681.jpg.chip.jpg


#### Model 2 - Hyperparameters

In [63]:
model = CNNRegression().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20
loss_fn = nn.MSELoss()
batch_sz = 100

#### Instantiating Train and Val Dataloaders

In [64]:
train_dataloader = CustomDataloader(dataframe=cnn_x_train, age=cnn_y_train_tensor, batch_size=batch_sz, randomize=True)
val_dataloader = CustomDataloader(dataframe=cnn_x_val, age=cnn_y_val_tensor, batch_size=batch_sz, randomize=False)

#### Model 2 - Training Loop

In [65]:
train_losses = []
val_losses = []
for epoch in tqdm.tqdm(range(epochs)):
    # Training step
    train_loss = cnn_train_step(model, train_dataloader, loss_fn, optimizer, device)
    train_losses.append(train_loss)
    
    # Validation step
    val_loss = cnn_val_step(model, val_dataloader, loss_fn, device)
    val_losses.append(val_loss)

    save_checkpoint(model, optimizer, epoch, f'model_checkpoints/cnn_checkpoint_epoch_{epoch}.pth')
save_checkpoint(model, optimizer, epochs, f'model_checkpoints/cnn_checkpoint_final.pth')

100%|██████████| 20/20 [07:22<00:00, 22.13s/it]


In [66]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\cnn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\cnn_val_losses.pt')




### Model 3 - Training Multi-Modal Network ###

#### Load in Train, Val, and Test sets for Model 3

In [67]:
directory = 'tensor_collection\\'

#load train and val csv files
mnn_x_train = pd.read_csv(directory + 'mnn_x_train.csv')
mnn_x_val = pd.read_csv(directory + 'mnn_x_val.csv')
mnn_y_train_tensor = torch.load(directory + 'mnn_y_train.pt')
mnn_y_val_tensor = torch.load(directory + 'mnn_y_val.pt')

print(type(mnn_x_train))
print(type(mnn_y_train_tensor))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


#### Model 3 - Hyperparameters

In [68]:
num_numerical_features = nn_x_train.shape[1] - 1 # -1 because we don't count the filename column

model = MultiModalNetwork(num_numerical_features).to(device)  # Adjust for the number of numerical features
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20 #after 10 epochs the model train and validation loss began to spike
loss_fn = torch.nn.MSELoss()
batch_sz = 100


#### Instantiating Train and Val Data loaders

In [69]:
train_dataloader = CustomDataloader(dataframe=mnn_x_train, age=mnn_y_train_tensor, batch_size=batch_sz, randomize=True)
val_dataloader = CustomDataloader(dataframe=mnn_x_val, age=mnn_y_val_tensor, batch_size=batch_sz, randomize=False)


#### Model 3 - Training Loop

In [70]:
train_losses = []
val_losses = []
for epoch in tqdm.tqdm(range(epochs)):
    # Training step
    train_loss = mmn_train_step(model, train_dataloader, loss_fn, optimizer, device)
    train_losses.append(train_loss)

    # Validation step
    val_loss = mmn_val_step(model, val_dataloader, loss_fn, device)
    val_losses.append(val_loss)

    checkpoint_filename = f'model_checkpoints/mmn_checkpoint_epoch_{epoch}.pth'
    save_checkpoint(model, optimizer, epoch, checkpoint_filename)
save_checkpoint(model, optimizer, epochs, f'model_checkpoints/mmn_checkpoint_final.pth')

100%|██████████| 20/20 [07:24<00:00, 22.21s/it]


In [71]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\mmn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\mmn_val_losses.pt')

